In [4]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, confusion_matrix

# --- Load data ---
df_clean = pd.read_csv("../multi_class_grid_clean.csv")

In [5]:
test_polo = "Belo Horizonte" 
train_df = df_clean[df_clean["Polo"] != test_polo]
test_df  = df_clean[df_clean["Polo"] == test_polo]

In [6]:
X_train = train_df.drop(columns=["target", "Polo"])
y_train = train_df["target"]
X_test  = test_df.drop(columns=["target", "Polo"])
y_test  = test_df["target"]

In [7]:
X_train_main, X_val, y_train_main, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

In [8]:
lgbm = LGBMClassifier(
    objective="binary",
    class_weight="balanced",  # handles imbalance automatically
    n_estimators=500,
    learning_rate=0.05,
    max_depth=-1,
    random_state=42,
    n_jobs=-1
)


In [10]:
from lightgbm import early_stopping, log_evaluation

lgbm.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric="auc",
    callbacks=[
        early_stopping(stopping_rounds=50),
        log_evaluation(period=50)
    ]
)


[LightGBM] [Info] Number of positive: 78205, number of negative: 477172
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.265988 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7773
[LightGBM] [Info] Number of data points in the train set: 555377, number of used features: 33
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
Training until validation scores don't improve for 50 rounds
[50]	valid_0's auc: 0.871686	valid_0's binary_logloss: 0.471316
[100]	valid_0's auc: 0.890977	valid_0's binary_logloss: 0.432393
[150]	valid_0's auc: 0.90269	valid_0's binary_logloss: 0.411624
[200]	valid_0's auc: 0.910301	valid_0's binary_logloss: 0.397791
[250]	valid_0's auc: 0.916074	valid_0's binary_logloss: 0.386803
[300]	valid_0's auc: 0.920685	valid_0's binary_logloss: 0.

LGBMClassifier(class_weight='balanced', learning_rate=0.05, n_estimators=500,
               n_jobs=-1, objective='binary', random_state=42)

In [11]:
y_proba = lgbm.predict_proba(X_test)[:, 1]
y_pred = (y_proba >= 0.5).astype(int)

auc = roc_auc_score(y_test, y_proba)
acc = accuracy_score(y_test, y_pred)

print(f"\n✅ Test Results for Polo '{test_polo}':")
print(f"AUC: {auc:.3f}")
print(f"Accuracy: {acc:.3f}")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


✅ Test Results for Polo 'Belo Horizonte':
AUC: 0.753
Accuracy: 0.781
[[125108  14671]
 [ 24741  15261]]
              precision    recall  f1-score   support

           0       0.83      0.90      0.86    139779
           1       0.51      0.38      0.44     40002

    accuracy                           0.78    179781
   macro avg       0.67      0.64      0.65    179781
weighted avg       0.76      0.78      0.77    179781

